# DEPRECATED

## This method of interacting with the Capella API has been deprecated in favor of the [Capella Console Client](https://github.com/capellaspace/console-client)

# Capella API: Search, Order, and Download Tutorial

In [ ]:
# Required libraries:
# requests
# json
# folium

In [ ]:
!pip install folium # skip this if library is already installed

Your username and password must be saved in a .json file named 'credentials.json' and formatted as follows.

{"username": "yourusername","password": "xxxxxxxxx"}

### Import required libraries, build a print utility function, assign API endpoints and load Credentials

In [ ]:
import requests
import json

# JSON utility function
def p(data):
    print(json.dumps(data, indent=2))

# Capella API endpoints
URL = 'https://api.capellaspace.com'
token = '/token'
collections = '/catalog/collections'
catsearch = '/catalog/search'
orders = '/orders/'

#Load username and password
with open('../credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']
username

### Get and Print Access Token

In [ ]:
#Get the token
r = requests.post(URL + token, 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))
accesstoken = r.json()["accessToken"]

print("Access Token: " + accesstoken)
# form the headers that will be used in other API calls
headers = {'Authorization':'Bearer ' + accesstoken}

### Print Available Collections

In [ ]:
# See what collections are available
# IMPORTANT: The Capella catalog organizes data into "collections" which are logical buckets.
#            Not to be confused with the collect which is the acquisition (which has a collect_id)
#
r = requests.get(URL + collections, headers=headers)

# Print the results
#p(r.json())

### Search the catalog for the desired collect(s)

In [ ]:
# Post search filters
# These are optional, but you will need to provide something find what you are interested in.
filters = {
    # lower left coodinate and upper right coordinate, in decimal degrees
    "bbox": [150.66736221313477, -33.62605502663527, 150.74323654174805, -33.56228145923039], 
    # collected time: start date/end date and time in YYYY-MM-DDTHH:MM:SSZ format
    "datetime": "2021-05-05T00:00:00Z/2022-05-05T16:30:00Z",  
    # overwrite the default pagination limit of 10, adjust as necessary
    "limit": 10, 
    # Optional filter: specify the desired collection such as capella-geo, capella-slc, capella-vessel-detection, etc.
    "collections": ["capella-geo"], 
    "sortby": "properties.datetime",
    "product": {
      "in": [
        "GEO",
        "SLC",
        "VS"
      ]
    },
    #
    # IF YOU KNOW THE collect_id. Then you can just grab it directly. 
    # See the Capella-API-task-monitor-accepted for how to get a collect id from a task
    #
    # "query": {
    #     "capella:collect_id": {
    #         "eq": "dd3d3be8-fb48-4ebc-afbf-d265f8effdd1"
    #     }
    # }
}
headers = {'Content-Type': 'application/json',
  'Accept': 'application/geo+json', 'Authorization':'Bearer ' + accesstoken}
r = requests.post(URL + catsearch, json=filters, headers=headers)
jresult = r.json()
print("Found: %s" % jresult['context']['matched'])
# Inspect the results
# p(jresults)

### Visualize Results on a Map

In [ ]:
# Display the results on a folium map
import folium

loc = [(filters["bbox"][1]+filters["bbox"][3])/2, (filters["bbox"][0]+filters["bbox"][2])/2] 
f = folium.Figure(width=700, height=200)  #sizing for the cell
m = folium.Map(location=loc,zoom_start=9).add_to(f) # create and initialize the map
folium.GeoJson(r.text).add_to(m) # add the bbox geometries to the map
m.fit_bounds(filters["bbox"])
m

### Make and Post an Order
By placing an order, you will then be granted access to download the assets (images or analytic products)

In [ ]:
# Make an Order
features = jresult["features"]
granulelist = []

# Loop over the features and get what you want from the response 
# and add to an array for an order
# Note: this is a combination of the Collection name and GranduleId 
#       (not the collect_id which represents the collected data instance. Collection is the catalog "bucket")
for f in features:
    item = {"CollectionId": f["collection"], "GranuleId": f["id"]}
    print(item)
    granulelist.append(item)
    break # let's just grab one for the example
    
myorder = {"Items": granulelist}
myorder

In [ ]:
# Post the order and inspect the result
r = requests.post(URL + orders, json=myorder, headers=headers)
p(r.json())

### Get the STAC records with the signed URLs using the /download endpoint, Print the Result

In [ ]:
myorderid = r.json()["orderId"]
r = requests.get(URL + orders + myorderid + '/download', headers=headers)
features = r.json()
for feature in features:
    signed_url = feature["assets"]["HH"]["href"]
    break  #just grabbing one for the example
signed_url
# p(r.json())

### Download the Results

In [ ]:
# Create a local file name using the granuleId - parsed from the signed_url
filename = signed_url[signed_url.rfind("/")+1:]
sep = "?"
tif_file_name = filename.split(sep, 1)[0]
tif_file_name

In [ ]:
# Download the file
import os
local_file_name = "%s/%s" % (os.getenv("HOME"), tif_file_name)
print("Downloading to: %s" % local_file_name)
print("...")
with requests.get(signed_url, stream=True) as result:
    result.raise_for_status()
    with open(local_file_name, 'wb') as f:
        for chunk in result.iter_content(chunk_size=10000000):
            f.write(chunk) 
print("Done!")